### This notebook is for fine-tuning BERT for sentiment analysis on the movie reviews dataset.
### In this example we use BCE loss. But the loss function can be changed to other loss functions, such as triplet loss, contrastive loss, etc.


In [1]:
import pandas as pd
file_path = "./all_train.csv"

df = pd.read_csv(file_path)

print(df.head())
print(df.shape)


                                         description    y
0  For a movie that gets no respect there sure ar...  1.0
1  Bizarre horror movie filled with famous faces ...  1.0
2  A solid, if unremarkable film. Matthau, as Ein...  1.0
3  It's a strange feeling to sit alone in a theat...  1.0
4  You probably all already know this by now, but...  1.0
(50000, 2)


In [2]:
import torch
from torch.nn import BCEWithLogitsLoss
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Load and preprocess data
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df['description'].tolist(), df['y'].tolist()

# Tokenize and encode the data
def encode_data(texts, labels, tokenizer, max_length=128):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    return TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels, dtype=torch.float))

# Fine-tune BERT
def fine_tune_bert(train_dataset, val_dataset, batch_size=16, epochs=3, learning_rate=2e-5, patience=2):
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    loss_fn = BCEWithLogitsLoss()
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    best_val_loss = float('inf')
    best_val_accuracy = 0
    early_stopping_counter = 0
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits.squeeze(), labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")
        
        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [b.to(device) for b in batch]
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = loss_fn(outputs.logits.squeeze(), labels)
                val_loss += loss.item()
                predicted = (outputs.logits.squeeze() > 0).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = correct / total
        print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_accuracy = val_accuracy
            early_stopping_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print(f"Early stopping triggered after epoch {epoch+1}")
                break
    
    # Load the best model
    model.load_state_dict(torch.load('best_model.pth'))
    return model, best_val_loss, best_val_accuracy, epoch + 1

In [3]:
# Load data
texts, labels = load_data(file_path)

In [4]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
# Encode data
dataset = encode_data(texts, labels, tokenizer)

In [6]:
# Split data into train, validation, and test
train_val_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_dataset, val_dataset = train_test_split(train_val_dataset, test_size=0.2, random_state=42)

### Fine tune the BERT model with different hyperparameters and save the best model


In [7]:
# Hyperparameters to experiment with
batch_sizes = [8, 16, 32]
epochs_list = [3, 4, 5]
learning_rates = [2e-5, 5e-5]

best_val_accuracy = 0
best_params = {}
best_overall_model = None

for batch_size in batch_sizes:
    for epochs in epochs_list:
        for lr in learning_rates:
            print(f"\nFine-tuning with batch_size={batch_size}, epochs={epochs}, learning_rate={lr}")
            model, val_loss, val_accuracy, epochs_run = fine_tune_bert(train_dataset, val_dataset, batch_size=batch_size, epochs=epochs, learning_rate=lr)
            
            print(f"Final Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")
            print(f"Epochs run: {epochs_run}")
            
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                best_params = {'batch_size': batch_size, 'epochs': epochs_run, 'learning_rate': lr}
                best_overall_model = model
            
            # Clear CUDA cache to free up memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

# Final evaluation on the test set
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'])
best_overall_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [b.to(best_overall_model.device) for b in batch]
        outputs = best_overall_model(input_ids, attention_mask=attention_mask)
        predicted = (outputs.logits.squeeze() > 0).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f"\nBest parameters: {best_params}")
print(f"Best validation accuracy: {best_val_accuracy:.4f}")
print(f"Test accuracy with best model: {test_accuracy:.4f}")

# Save the best model
best_overall_model.save_pretrained('best_fine_tuned_bert_sentiment')
tokenizer.save_pretrained('best_fine_tuned_bert_sentiment')
print("Fine-tuning complete. Best model saved.")


Fine-tuning with batch_size=8, epochs=3, learning_rate=2e-05


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3: 100%|██████████| 4000/4000 [02:30<00:00, 26.65it/s]


Epoch 1/3, Loss: 0.3106
Validation Loss: 0.2835, Accuracy: 0.8824


Epoch 2/3: 100%|██████████| 4000/4000 [02:28<00:00, 26.91it/s]


Epoch 2/3, Loss: 0.1575
Validation Loss: 0.2786, Accuracy: 0.8966


...

Epoch 1/5, Loss: 0.3209
Validation Loss: 0.2721, Accuracy: 0.8904


Epoch 2/5: 100%|██████████| 1000/1000 [01:25<00:00, 11.63it/s]


Epoch 2/5, Loss: 0.1657
Validation Loss: 0.2997, Accuracy: 0.8942


Epoch 3/5: 100%|██████████| 1000/1000 [01:26<00:00, 11.61it/s]


Epoch 3/5, Loss: 0.0637
Validation Loss: 0.3514, Accuracy: 0.8941
Early stopping triggered after epoch 3
Final Validation Loss: 0.2721, Accuracy: 0.8904
Epochs run: 3

Best parameters: {'batch_size': 8, 'epochs': 3, 'learning_rate': 2e-05}
Best validation accuracy: 0.8966
Test accuracy with best model: 0.8973
Fine-tuning complete. Best model saved.


### Predict the sentiments for some example reviews

In [8]:
from torch.nn import functional as F

def load_model(model_path):
    # Load the fine-tuned model
    model = BertForSequenceClassification.from_pretrained(model_path)
    # Load the tokenizer
    tokenizer = BertTokenizer.from_pretrained(model_path)
    return model, tokenizer

def predict_sentiment(text, model, tokenizer):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    
    # Move inputs to the same device as the model
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Set the model to evaluation mode
    model.eval()
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    # Apply sigmoid to get probability
    probs = F.sigmoid(logits)
    
    # Get the predicted class (0 for negative, 1 for positive)
    predicted_class = (probs > 0.5).int().item()
    
    # Get the probability of the predicted class
    confidence = probs.item() if predicted_class == 1 else 1 - probs.item()
    
    return predicted_class, confidence

# Load the model
model_path = 'best_fine_tuned_bert_sentiment'
model, tokenizer = load_model(model_path)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Example usage
texts = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "The film was a complete disaster. Terrible acting and a nonsensical plot.",
    "It was an okay movie. Nothing special, but not terrible either."
]

for text in texts:
    sentiment, confidence = predict_sentiment(text, model, tokenizer)
    sentiment_label = "Positive" if sentiment == 1 else "Negative"
    print(f"Text: {text}")
    print(f"Sentiment: {sentiment_label}")
    print(f"Confidence: {confidence:.4f}")
    print()

Text: This movie was absolutely fantastic! I loved every minute of it.
Sentiment: Positive
Confidence: 0.9910

Text: The film was a complete disaster. Terrible acting and a nonsensical plot.
Sentiment: Negative
Confidence: 0.9982

Text: It was an okay movie. Nothing special, but not terrible either.
Sentiment: Positive
Confidence: 0.5151

